In [ ]:
import os # import the os module

from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_react_agent,AgentOutputParser
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

# Set your Groq API key as an environment variable
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual API key

# Initialize ChatGroq model (using Llama3-8B model)
model = ChatGroq(model="llama3-8b-8192")

# Set up Neo4j Graph connection
graph = Neo4jGraph(
    url="neo4j+s://b3c3b7e7.databases.neo4j.io",
    username="neo4j",
    password="XZ7Ly1hqINRKplRVxuZTfVG-V1oqYMYZQAK0p46XcgU"
)

# Define the prompt template for conversation
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a Neo4j expert having a conversation about how to create Cypher queries",
        ),
        ("human", "{input}"),
    ]
)

# Create Cypher chat function using StrOutputParser
cypher_chat = prompt | model | StrOutputParser()

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

# Define the tools for the agent
tools = [
    Tool.from_function(
        name="Cypher Support",
        description="For when you need to talk about Cypher queries.",
        func=cypher_chat.invoke,
    )
]

# Pull agent prompt from hub (you can modify this based on your custom agent prompt if necessary)
agent_prompt = hub.pull("hwchase17/react-chat")

# Create the agent
agent = create_react_agent(model, tools, agent_prompt)

# Set up the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Wrap the agent executor with message history
cypher_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Start a loop to interact with the agent
while True:
    q = input("> ")

    # Invoke the agent with the provided query and session ID
    response = cypher_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )
    
    # Print the response from the agent
    print(response["output"])


INSTALL LIBRARY
pip install langchain
pip install langchain-core
pip install langchain-community
pip install neo4j
pip install langchain-groq
